In [78]:
# %%capture
# !pip install langchain==0.3.8
# !pip install langchain_community #0.3.9
# !pip install tiktoken
# !pip install faiss-cpu
# !pip install gradio
# !pip install Pillow
# !pip install transformers #4.46.2
# !pip install openai==0.28
# # !pip show transformers #Version: 0.3.8
# !pip show openai #1.57.0 => 0.28
# !pip show requests
# !pip install presidio_analyzer
# !pip install presidio_anonymizer
!pip install transformers spacy presidio-analyzer presidio-anonymizer
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [76]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import userdata
import os
import openai

from transformers import pipeline
import spacy
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from spacy.matcher import PhraseMatcher
import json
import re

from langchain import hub
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import create_react_agent, initialize_agent, Tool, ZeroShotAgent, load_tools, AgentExecutor, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.prompts import PromptTemplate

from sentence_transformers import SentenceTransformer, util
import pandas as pd
import openai
import json
import gradio as gr
from google.colab import userdata
import os
import re
import ast
import json
import requests
from PIL import Image
from io import BytesIO
from transformers import CLIPTokenizer
import torch

In [4]:
# _------ OPENAI API KEY ------------
api_key = userdata.get('OA_API')
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

# -----  Models used ---------------
np.random.seed(42)
llm_gpt4 = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=os.environ["OPENAI_API_KEY"])
llm_gpt3 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

model_transformer = SentenceTransformer('clip-ViT-B-32')
tokenizer_clip = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

!pip show openai transformers
# ------------ Read DATA ------
data = pd.read_csv("/content/drive/MyDrive/GenAI@IISC_Group6/DB_Amazon_cellphone_features.csv")
data["ProductDetailsFineTuned"] = "-ASIN : " + data["parent_asin"] + "\n" + data["ProductDetailsFineTuned"]
features_list = data["ProductDetailsFineTuned"].tolist()
data.head()

<ipython-input-4-b44bd8aa5ef3>:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_gpt4 = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=os.environ["OPENAI_API_KEY"])


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

0_CLIPModel/config.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 
---
Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,Details,ProductDetails,ProductDetailsFineTuned
0,Cell Phones & Accessories,SAMSUNG Galaxy A52A525F128GB 6GB RAMFactory Un...,4.3,685,4G Bands123457812172026283238404166.Cameras64 ...,Behold the comfortablesleek curves on Galaxy A...,199.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Samsung Galaxy A52 - Honest Review...,SAMSUNG,"['Cell Phones & Accessories', 'Cell Phones']",{'Package Dimensions': '7.68 x 4.76 x 2.09 inc...,B098BK2SJ5,NaN,NaN,NaN,Mobile ASIN : B098BK2SJ5\nMobile Brand : samsu...,- Brand Name: Samsung\n- Mobile Name: Samsung ...,-ASIN : B098BK2SJ5\n- Brand Name: Samsung\n- M...
1,Cell Phones & Accessories,Moto G Power20223-Day BatteryUnlockedMade for ...,4.2,655,Carrier compatibilityAT&TVoLTEVerizonVoLTET Mo...,Moto G Power 2022Up to three days of battery l...,129.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Moto G 5g Watch Before You Get It ...,Motorola,"['Cell Phones & Accessories', 'Cell Phones']",{'Product Dimensions': '6.57 x 3.03 x 0.35 inc...,B0BL9TKSBL,NaN,NaN,NaN,Mobile ASIN : B0BL9TKSBL\nMobile Brand : motor...,- Brand Name: Motorola\n- Mobile Name: Moto G ...,-ASIN : B0BL9TKSBL\n- Brand Name: Motorola\n- ...
2,Cell Phones & Accessories,Apple iPhone 14 Plus512GB(PRODUCTRedUnlocked (...,4.5,225,6.7-inch Super Retina XDR displayadvanced came...,The iPhone 14 Plus is an excellent choice if y...,909.89,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'iPhone 14 vs iPhone 14 Plus - Size...,Amazon Renewed,"['Cell Phones & Accessories', 'Cell Phones']","{'Package Dimensions': '8 x 5 x 3 inches', 'It...",B0BY5V82W4,NaN,NaN,NaN,Mobile ASIN : B0BY5V82W4\nMobile Brand : amazo...,- Brand Name: Amazon Renewed\n- Mobile Name: A...,-ASIN : B0BY5V82W4\n- Brand Name: Amazon Renew...
3,Cell Phones & Accessories,SAMSUNG Galaxy S23 Cell PhoneFactory Unlocked ...,4.4,64,CAPTURE THE NIGHT IN LOW LIGHTWhether youâ€™re...,Meet Galaxy S23the phone takes you out of the ...,759.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Do this as soon as you get your S2...,SAMSUNG,"['Cell Phones & Accessories', 'Cell Phones']",{'Package Dimensions': '6.38 x 3.46 x 1.06 inc...,B0BXKWGCSW,NaN,NaN,NaN,Mobile ASIN : B0BXKWGCSW\nMobile Brand : samsu...,- Brand Name: Samsung\n- Mobile Name: Samsung ...,-ASIN : B0BXKWGCSW\n- Brand Name: Samsung\n- M...
4,Cell Phones & Accessories,SAMSUNG Galaxy S22Plus 256GB Factory Unlocked ...,4.4,136,Renewed conditionMay have few minor scuffs or ...,Meet the SAMSUNG Galaxy S22the smartphone that...,493.00,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Samsung S22 Ultra', 'url': 'https:...",Amazon Renewed,"['Cell Phones & Accessories', 'Cell Phones']","{'Product Dimensions': '3 x 0.3 x 6 inches', '...",B0B4VCQ7MB,NaN,NaN,NaN,Mobile ASIN : B0B4VCQ7MB\nMobile Brand : amazo...,- Brand Name: Amazon Renewed\n- Mobile Name: S...,-ASIN : B0B4VCQ7MB\n- Brand Name: Amazon Renew...


In [5]:
review_data  = pd.read_csv("/content/drive/MyDrive/GenAI@IISC_Group6/DB_Amazon_cellphone_reviews.csv")
review_data["review_length"] = review_data["text"].str.split().str.len()
review_data["title_length"] = review_data["title"].str.split().str.len()
review_data = review_data[review_data["review_length"]>10]
review_data["review_text"] = "- " + review_data["text"]
review_data.head()

<ipython-input-5-a062846505a8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_data["review_text"] = "- " + review_data["text"]


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,review_length,title_length,review_text
0,1,Too many ads. Can't hook up to verizon,Too many ads. Can't set lockscreen image becau...,[],B075QWLV3J,B075QWLV3J,AHATA6X6MYTC3VNBFJ3WIYVK257A,2018-04-07 17:36:18.834,0,False,87.0,8.0,- Too many ads. Can't set lockscreen image bec...
1,5,Best phone yet,I love this phone it has everything and it doe...,[],B08P54F6HB,B0B8PDLMMS,AFAMOLAPKR3M2NPTMB362T2Q5RBQ,2021-09-08 23:51:02.923,1,True,14.0,3.0,- I love this phone it has everything and it d...
3,5,LG phone,I am very happy with my new phone. I love the ...,[],B009LRNLW2,B00MU042H0,AEPLHPUQ6UTBO2PUZGM3CQ6YG5WQ,2014-02-13 13:11:36.000,0,True,30.0,2.0,- I am very happy with my new phone. I love th...
4,5,Very nice phone,The renewed phone arrived in perfect condition...,[],B082T4F34B,B09JJCXC2M,AFOMMZPFMNVCAC6VYYGQPJSRZFTQ,2021-09-19 13:25:13.275,0,True,81.0,3.0,- The renewed phone arrived in perfect conditi...
5,4,"Clean, unlocked and as described.","Love the Size, physical and memory. Not using ...",[],B07P6SWG7T,B09JJCXC2M,AFHFWRPC2P2R4FISMWNGX5E4A7TA,2020-05-19 20:45:45.585,0,True,59.0,5.0,"- Love the Size, physical and memory. Not usin..."


In [7]:
#------------ FEATURE AGENT ----------
def extract_input_features(text, model="gpt-3.5-turbo"):
    """ Extract features using OpenAI's chat-based models."""
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an assistant that extracts key features of products."},
            {"role": "user", "content": f"Extract and list the key features which are available from ⁠Brand Name,⁠ ⁠⁠Mobile Name, ⁠⁠Processor, ⁠Storage, ⁠RAM, ⁠Camera, ⁠⁠Price, Sim Type, ⁠⁠Battery Life, ⁠Average Rating, ⁠⁠Count of Rating and ⁠Screen Details of the following product:\n Text : {text}.\n Make sure to provide information only from Text\n If feature details are not available in Text, Provide NA only with respect to that feature."}
        ],
        max_tokens=300,
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

def recommend_similar_features(input_feature_text):
    """
    Recommend products with similar features """
    vectorizer = TfidfVectorizer()
    feature_vectors = vectorizer.fit_transform(
        ["\n".join([v
                    for v in extract_input_features(input_feature_text).split("\n")
                    if "NA" not in v and "Not" not in v])] + features_list
        )

    # Compute cosine similarity
    similarity_scores = cosine_similarity(feature_vectors[0:1], feature_vectors[1:]).flatten()

    # Sort products by similarity
    sorted_indices = np.argsort(-similarity_scores)
    recommendations = pd.DataFrame([
        {
            "features": features_list[idx],
            "similarity_score": similarity_scores[idx]
        }
        for idx in sorted_indices
    ])
    recommendations = recommendations[recommendations["similarity_score"]>=0.1].sort_values(by="similarity_score",ascending=False).head(5)["features"].values
    return "\n\n".join(recommendations)


features_tool = Tool(
    name = "RecommendSimilarFeatures",
    func = recommend_similar_features,
    description = (
        "Recommend similar products based on features. "
        "Provide ASIN, input features and a list of product descriptions."
    )
)

recommendation_agent = initialize_agent(
    tools=[features_tool],
    llm= llm_gpt4,
    verbose=True,
    memory=ConversationBufferMemory(),
    handle_parsing_errors=True
)

input_product = "Suggest me mobile with Snapdragon 690 "
response = recommendation_agent.run(input_product)
print(response)



> Entering new AgentExecutor chain...
I need to find mobile phones that are equipped with the Snapdragon 690 processor.  
Action: RecommendSimilarFeatures  
Action Input: "mobile with Snapdragon 690"  
Observation: -ASIN : B07XYKK71P
- Brand Name: OnePlus
- Mobile Name: OnePlus Nord N10 5G Global/EU Version
- Processor: Qualcomm Snapdragon 690 5G
- Storage: 128GB UFS 2.1, expandable up to 512GB
- RAM: 6GB LPDDR4X
- Camera: 
  - Rear Camera: 64MP main + 8MP ultra-wide + 2MP macro + 2MP monochrome
  - Front Camera: 16MP
- Price: $299.99
- Sim Type: GSM Unlocked (Works with T-Mobile, AT&T, Metro, etc.)
- Summary of Description: Great budget Android smartphone with 6.49-inch FHD display, 90Hz refresh rate, quad-camera setup, Snapdragon 690 processor, and 4300mAh battery with Warp Charge 30T fast charging.
- Battery Life: 4300mAh (non-removable)
- Average Rating: 4.1
- Count of Rating: 48
- Screen Details: 6.49 inches, FHD (1080x2400), 90Hz refresh rate, IPS LCD capacitive touchscreen, 40

In [8]:
# ------------- REVIEW ANALYSIS AGENT ------------
def summarize_reviews(reviews_text, model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes reviews."},
            {"role": "user", "content": f"Summarize the following reviews for a product in key point as list and be specific with pros and cons :\n{reviews_text}"}
        ],
        max_tokens=150,
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


summarize_reviews_tool = Tool(
    name="SummarizeReviews",
    func=summarize_reviews,
    description=(
        "This tool summarizes multiple reviews for a specific product into a brief summary. "
        "Provide all the reviews as input."
    )
)

summarize_reviews_agent = initialize_agent(
    tools=[summarize_reviews_tool],
    llm= llm_gpt3,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=ConversationBufferMemory(max_tokens=8000),
    handle_parsing_errors=True
)

def get_reviews(reviews_data, asin):
  reviews = "\n".join(
      reviews_data[reviews_data["parent_asin"]==asin].sort_values("timestamp")["review_text"])
  return reviews


In [11]:
# ---------- PRICE AGENT ----------
def create_price_agent_title(data,column1,column2,column3):
    text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=20,separator='\n\n')
    texts = text_splitter.create_documents(data[column1].astype(str).tolist())
    print(texts)

    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)
    docsearch = FAISS.from_documents(texts, embeddings)
    qa = RetrievalQA.from_chain_type(llm= llm_gpt4, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k":6}))
    tool = Tool(
        name=f"{column1}",
        func= qa.run,
        description=f"""Useful for when you need to answer questions about
        the {column1} column in the Amazon Cell Phones dataset.
        The price {column2} for each phone is listed next to the PhoneName={column3}
        Only fetch answer from document search the price about cell phones {column2},in the context of cell phone price
        not about other products or off-topic questions.
        Output only Phones with matched criteria, do not give output of unmatched phones.""",
    )
    return tool

def create_price_agent_asin(data,column1,column2,column3):
    text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=20,separator='\n\n')
    texts = text_splitter.create_documents(data[column1].astype(str).tolist())
    print(texts)

    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)
    docsearch = FAISS.from_documents(texts, embeddings)
    qa = RetrievalQA.from_chain_type(llm= llm_gpt4, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k":6}))
    tool = Tool(
        name=f"{column1}",
        func= qa.run,
        description=f"""Useful for when you need to answer questions about
        the {column1} column in the Amazon Cell Phones dataset.
        The price {column2} for each phone is listed next to the parent_asin={column3}. parent_asin is uniqueID of each phone.
        Only fetch answer from document search the price about cell phones {column2},in the context of cell phone price
        not about other products or off-topic questions.
        Output only parent_asin with matched criteria, do not give output of unmatched. Outout should be ASIN=parent_asin:matched criteria; for each phone, do not add any additional text.""",
    )
    return tool

reAct_prompt = hub.pull("hwchase17/react")

data['price_search_title'] = 'Phone=' + data['title'] + ",Price=" + data['price'].astype(str)
price_title_tool = create_price_agent_title(data,column1="price_search_title",column2="price", column3= "title")

data['price_search_asin'] = "parent_asin=" + data['parent_asin'] + ",Price=" + data['price'].astype(str)
price_asin_tool = create_price_agent_asin(data,column1="price_search_asin",column2="price",column3="parent_asin")

# Create the agent
price_agent = create_react_agent(
    llm= llm_gpt4,
    tools=[price_title_tool, price_asin_tool],
    prompt= reAct_prompt,
    stop_sequence=True,
)
price_agent_executor = AgentExecutor.from_agent_and_tools(
    agent = price_agent,
    tools = [price_title_tool, price_asin_tool],
    verbose = True,
)


# ________________TESTING PURPOSE ONLY____________
# user_query = "parent_asin B07XY8YFQQ, B07XYKK71P filter products with price under 250$  "
# user_query = "suggest phones with price under 250$. Suggest 5 phones "
# response = price_agent_executor.invoke({"input": user_query})
# print(response["output"])

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[Document(metadata={}, page_content='Phone=SAMSUNG Galaxy A52A525F128GB 6GB RAMFactory Unlocked (GSM ONLYNot Compatible with Verizon/Sprint/BoostInternational Model (Awesome Purple),Price=199.99'), Document(metadata={}, page_content='Phone=Moto G Power20223-Day BatteryUnlockedMade for US by Motorola4/64GB50 MP CameraDark Grove,Price=129.99'), Document(metadata={}, page_content='Phone=Apple iPhone 14 Plus512GB(PRODUCTRedUnlocked (Renewed),Price=909.89'), Document(metadata={}, page_content='Phone=SAMSUNG Galaxy S23 Cell PhoneFactory Unlocked Android Smartphone256GB Storage50MP CameraNight ModeLong Battery LifeAdaptive DisplayUS Version2023Phantom Black,Price=759.99'), Document(metadata={}, page_content='Phone=SAMSUNG Galaxy S22Plus 256GB Factory Unlocked SM-S906U1 Pink Gold (Renewed),Price=493.0'), Document(metadata={}, page_content='Phone=ZTE Cymbal Z-320 Flip Phone UNLOCKED (T-Mobile),Price=89.0'), Document(metadata={}, page_content='Phone=Apple iPod Touch 16GB (5th GenerationSpace Gra

<ipython-input-11-fa117cc5f499>:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


[Document(metadata={}, page_content='parent_asin=B098BK2SJ5,Price=199.99'), Document(metadata={}, page_content='parent_asin=B0BL9TKSBL,Price=129.99'), Document(metadata={}, page_content='parent_asin=B0BY5V82W4,Price=909.89'), Document(metadata={}, page_content='parent_asin=B0BXKWGCSW,Price=759.99'), Document(metadata={}, page_content='parent_asin=B0B4VCQ7MB,Price=493.0'), Document(metadata={}, page_content='parent_asin=B06WGL82S6,Price=89.0'), Document(metadata={}, page_content='parent_asin=B00VQQ294C,Price=205.0'), Document(metadata={}, page_content='parent_asin=B07L6QY1TJ,Price=94.99'), Document(metadata={}, page_content='parent_asin=B09JFJ1Q5C,Price=678.39'), Document(metadata={}, page_content='parent_asin=B09X9FC88M,Price=769.0'), Document(metadata={}, page_content='parent_asin=B08DXKVYQK,Price=129.99'), Document(metadata={}, page_content='parent_asin=B0748NBWDP,Price=109.99'), Document(metadata={}, page_content='parent_asin=B0BGYC7HYQ,Price=849.0'), Document(metadata={}, page_cont

In [33]:
# ------------- IMAGE AGENT -------------

# prompt: prompt: display gradio interface to accept input_user_query and output format as above collected ASIN number as header and dictionary value in square block with black border. show image from imge.png under each product extracted.
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import openai
import glob

# Set your OpenAI API key
#openai.api_key = "YOUR_API_KEY"

class DalleImageCustomizer:
    def __init__(self, mobile_name, color=None):
        """
        Initialize the customizer with the mobile name and desired color.

        Args:
            mobile_name (str): Name of the mobile device.
            color (str, optional): Desired color to modify the image to. Defaults to "white".
        """
        self.file_path = "/content/Amazon_cellphone1_updated.csv"
        self.mobile_name = mobile_name
        self.color = color if color else "white"
        # self.image_url = self._fetch_image_url()

    def _fetch_image_url(self):
        """
        Fetch the image URL for the given mobile name from the CSV file.
        If no URL is found, return None.

        Returns:
            str: The image URL if found, None otherwise.
        """
        try:
            df = pd.read_csv(self.file_path)
            row = df[df["title"].str.contains(self.mobile_name, case=False, na=False)]
            if not row.empty:
                image_url = row.iloc[0]["large_image_url"]
                if pd.isnull(image_url) or image_url.strip() == "":
                    return None
                return image_url
            else:
                return None
        except Exception as e:
            print(f"Error reading CSV file: {e}")
            return None

    def fetch_image_from_url(self):
        """
        Fetches an image from the retrieved URL.

        Returns:
            PIL.Image.Image: The image object if fetched successfully, None otherwise.
        """
        if not self.image_url:
            return None
        try:
            response = requests.get(self.image_url)
            if response.status_code == 200:
                return Image.open(BytesIO(response.content)).convert("RGBA")
            else:
                print(f"Failed to fetch image from URL: {self.image_url}")
                return None
        except Exception as e:
            print(f"Error fetching image: {e}")
            return None

    def modify_image_with_dalle(self, input_image=None):
        """
        Modify the input image using DALLE to change the color of the mobile's backside.
        If no input image is provided, a random image will be generated.

        Args:
            input_image (PIL.Image.Image, optional): The input image.

        Returns:
            PIL.Image.Image: The customized image if successful, None otherwise.
        """
        try:
            prompt = (
                f"Create an image of a {self.mobile_name} with a {self.color} backside, "
                f"while keeping all other features such as the screen, camera, and dimensions intact. "
                f"Use the mobile as the main reference for features except for the backside color."
            )

            if not input_image:
                # Generate a random image using DALL-E based on the prompt
                print("Generating a new image using DALL-E...")
                dalle_response = openai.Image.create(
                    prompt=prompt,
                    n=1,
                    size="512x512",
                )
                output_image_url = dalle_response["data"][0]["url"]
                response = requests.get(output_image_url)
                print("Response : ", response.status_code)
                if response.status_code == 200:
                    return Image.open(BytesIO(response.content))#.convert("RGBA")
                else:
                    print("Failed to fetch DALL-E generated image.")
                    return None
            else:
                # Modify the input image with DALL-E
                prompt = (
                    f"Take the provided image of a {self.mobile_name} and modify it to have a {self.color} backside, "
                    f"while keeping all other features such as the screen, camera, and dimensions intact. "
                    f"Use the input image as a reference for all features except the backside color. Just return the image"
                )

                # Convert input image to bytes
                input_image_bytes = BytesIO()
                input_image.save(input_image_bytes, format="PNG")
                input_image_bytes.seek(0)

                dalle_response = openai.Image.create_edit(
                    image=input_image_bytes.read(),
                    prompt=prompt,
                    n=1,
                    size="512x512",
                )

                # Fetch the customized image URL
                output_image_url = dalle_response["data"][0]["url"]
                response = requests.get(output_image_url)
                if response.status_code == 200:
                    return Image.open(BytesIO(response.content)).convert("RGBA")
                else:
                    print("Failed to fetch edited image from DALL-E.")
                    return None
        except Exception as e:
            print(f"Error during image modification: {e}")
            return None

    def display_image(self):
        """
        Displays only the output image.

        Args:
            output_image (PIL.Image.Image): The customized output image.
        """
        if self.output_image:
            plt.imshow(self.output_image)
            plt.title(self.mobile_name)
            plt.axis("off")
            plt.show()
        else:
            print("No output image to display.")

    def process(self):
        """
        Processes the image customization workflow.
        """
        # Step 1: Fetch input image
        # input_image = self.fetch_image_from_url()

        # Step 2: Modify the image with DALLE
        self.output_image = None
        # if input_image:
        #     self.output_image = self.modify_image_with_dalle(input_image)
        # else:
        # If no input image exists, generate the customized image directly
        self.output_image = self.modify_image_with_dalle()

        # Step 3: Display the output image
        # self.display_image(self.output_image)

        # Step 4: Save the output image
        if self.output_image:
            self.output_image.save(f"images/{self.mobile_name}.png")
            print(f"Output image saved as images/{self.mobile_name}.png.")
        else:
            print("No output image generated.")


def image_generator(prompt: str):
  folder_name = "images"
  # Check if the folder exists
  if os.path.exists(folder_name):
      # If the folder exists, delete all .png files in the folder
      png_files = glob.glob(os.path.join(folder_name, "*.png"))
      for file in png_files:
          os.remove(file)
  else:
      # If the folder does not exist, create it
      os.makedirs(folder_name)

  modified_prompt = recommend_similar_features(prompt)
  filtered_mobile_name = prompt.split(",")
  print("filtered_mobile_name : ",filtered_mobile_name)
  mobile_dict = {}
  for mobile_name in filtered_mobile_name:
    mobile_dict[mobile_name] = DalleImageCustomizer(mobile_name)
    mobile_dict[mobile_name].process()
    # mobile_dict[mobile_name].display_image()
  return mobile_dict


generator_tool = Tool(
  name = "ImageGenerator",
  func = image_generator,
  description = (
      "Generate the images."
  )
)

generator_agent = initialize_agent(
    tools=[generator_tool],
    llm= llm_gpt4,
    verbose=True,
    memory=ConversationBufferMemory(),
    handle_parsing_errors=True
)


summary_agent = initialize_agent([features_tool], llm_gpt4, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)

In [65]:
class GuardRail4LLM:
    def __init__(self, pii_types=None, harmful_categories=None, topics=None, custom_harmful_keywords=None):
        # Initialize default configurations
        self.pii_types = pii_types or [
            "PERSON", "PHONE_NUMBER", "EMAIL_ADDRESS", "ADDRESS", "DATE_OF_BIRTH", "CREDIT_CARD",
            "SOCIAL_SECURITY_NUMBER", "PASSPORT", "DRIVER_LICENSE", "BANK_ACCOUNT", "IP_ADDRESS",
            "USER_NAME", "PASSWORD", "MOTHER_MAIDEN_NAME", "NATIONAL_ID", "BIOMETRIC_DATA",
            "HEALTH_INFORMATION", "GEOLOCATION", "RACE", "GENDER", "SEXUAL_ORIENTATION", "RELIGION",
            "POLITICAL_BELIEFS", "ETHNICITY", "MARITAL_STATUS", "EMPLOYMENT_STATUS", "EDUCATION",
            "FINANCIAL_INFORMATION", "TRAVEL_HISTORY", "PURCHASE_HISTORY", "LICENSE_PLATE",
            "VOTER_REGISTRATION", "MILITARY_SERVICE", "CRIMINAL_RECORD", "ACCOUNT_NUMBER", "DEVICE_ID",
            "USER_AGENT", "ONLINE_IDENTIFIER", "IP_ADDRESS_V6", "WEB_COOKIE", "LOCATION_COORDINATES"
        ]
        self.harmful_categories = harmful_categories or [
            "Toxicity", "Hate Speech", "Self-harm", "Abuse", "Harassment", "Violence", "Bullying", "Discrimination",
            "Profanity", "Sexual Content", "Extremism", "Terrorism"
        ]
        self.topics = topics or ["products", "mobile phones"]
        self.custom_harmful_keywords = custom_harmful_keywords or ["explosives", "bomb", "terrorism", "violence"]

        # Initialize Presidio Analyzer and Anonymizer
        self.analyzer = AnalyzerEngine()
        self.anonymizer = AnonymizerEngine()

        # Initialize SpaCy for topic detection
        self.nlp = spacy.load("en_core_web_md")
        self.matcher = PhraseMatcher(self.nlp.vocab)
        self._add_topics_to_matcher()

        # Initialize Hugging Face pipeline for content moderation (zero-shot classification)
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    def _add_topics_to_matcher(self):
        """Add topics to the SpaCy matcher."""
        for topic in self.topics:
            pattern = self.nlp.make_doc(topic)
            self.matcher.add(topic, [pattern])

    def detect_pii(self, text, pii_types=None):
        """Detect PII elements in the text using Presidio."""
        pii_types = pii_types or self.pii_types
        results = self.analyzer.analyze(text=text, language="en", entities=pii_types)
        return results

    def mask_pii(self, text, pii_types=None):
        """Mask detected PII elements in the text."""
        pii_results = self.detect_pii(text, pii_types)
        masked_text = text
        for result in sorted(pii_results, key=lambda x: x.start, reverse=True):
            masked_text = masked_text[:result.start] + "[MASKED]" + masked_text[result.end:]
        return masked_text

    def detect_topics(self, text):
        """Detect topics in the text using spaCy's PhraseMatcher."""
        doc = self.nlp(text)
        matches = self.matcher(doc)
        matched_topics = [self.nlp.vocab.strings[match_id] for match_id, start, end in matches]
        return matched_topics

    # def check_topic_alignment(self, detected_topics, allowed_topics):
    #   if not detected_topics:  # No topics detected
    #     return "Off-topic", []
    #   off_topic = [topic for topic in detected_topics if topic not in allowed_topics]
    #   if off_topic or not any(topic in allowed_topics for topic in detected_topics):
    #     return "Off-topic", off_topic
    #   return "On-topic", detected_topics
    def moderate_content(self, text):
        """Moderate content using Hugging Face's zero-shot classification and custom harmful keywords."""
        classification = self.classifier(text, candidate_labels=self.harmful_categories)
        top_label = classification["labels"][0]
        top_score = classification["scores"][0]

        # Check for harmful keywords (e.g., explosives, bomb, terrorism)
        if any(keyword in text.lower() for keyword in self.custom_harmful_keywords):
            return "HARMFUL_CONTENT"

        # If harmful content is detected by model
        if top_score > 0.7 and top_label in self.harmful_categories:
            return "HARMFUL_CONTENT"

        # Return "SAFE" if no harmful content is detected
        return "SAFE"

    def process_text(self, text, pii_types=None, allowed_topics=None):

          """Process text by combining PII masking, topic detection, and moderation."""
          # Mask PII
          masked_text = self.mask_pii(text, pii_types)

          # Detect topics
          detected_topics = self.detect_topics(masked_text)

          # Check topic alignment
          topic_status, topics_detected = self.check_topic_alignment(detected_topics, allowed_topics)

          # Moderate content
          moderation = self.moderate_content(masked_text)

          result = {
              "original_text": text,
              "masked_text": masked_text,
              "topics": topics_detected,
              "topic_status": topic_status,
              "moderation": moderation,
          }

          # Return result as a JSON formatted string
          return json.dumps(result, indent=4)

    def batch_process(self, texts, pii_types=None, allowed_topics=None):
          """Batch process multiple texts."""
          return [self.process_text(text, pii_types, allowed_topics) for text in texts]

    def check_topic_alignment(self, detected_topics, allowed_topics, threshold=1.0):
      """
      Check if the detected topics align with the allowed topics based on semantic similarity.
      :param detected_topics: List of topics detected in the text.
      :param allowed_topics: List of allowed topics.
      :param threshold: Similarity threshold to determine if topics are "similar".
      """

      if not detected_topics:  # No topics detected
          return "Off-topic", []

      off_topic = []
      on_topic = []

      allowed_topics_lower = [self.nlp(topic.lower()) for topic in allowed_topics]

      for topic in detected_topics:
          topic_doc = self.nlp(topic.lower())
          # Check if the topic matches any allowed topic based on similarity
          is_similar = any(
              topic_doc.similarity(allowed_topic) >= threshold
              for allowed_topic in allowed_topics_lower
          )
          if is_similar:
              on_topic.append(topic)
          else:
              off_topic.append(topic)

      if on_topic:
          return "On-topic", on_topic
      return "Off-topic", off_topic


In [93]:
def route_agents(user_query):
    routing_prompt = (
        "You are an intelligent assistant that routes user queries to specific agents. "

        "Based on the requested features in user query determine which agent should handle it:\n"
        f"User Query: {user_query}\n"

        " Include features_tool if features {Brand Name, RAM, Storage, Camera, Price, Sim Type, Battery Life, Screen Details, Processor} found :\n"  # determine which agent should handle it
        " Also include price_tool if Price or dollar or $ details or range found, ; "

        "Available agents are:\n"
        "1. features_tool\n"
        "2. price_tool\n"
        "3. ImageSearchTool\n"
        "Respond with all the names of appropriate agents in python list."
    )
        # Modified section to fix the issue
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": routing_prompt}],
    )
    return response.choices[0].message.content.strip()

def filter_price_products_byAsin(product_asins, max_price, min_price):
  filtered_data = data[(data["parent_asin"].isin(product_asins)) & (data["price"] < max_price) & (data["price"] > min_price)]
  return filtered_data

def remove_price_line(text):
  cleaned_text = "\n".join([txt for txt in text.split("\n") if "Price" not in txt])
  return cleaned_text

# Main function for handling user input and routing
def orchestrate_agents(user_query):
    filtered_products = []
    agent_output = []
    reviews_summary = {}
    feature_agent_response = price_agent_response = summary_agent_response = ""
    max_token_limit=8000

    prompt_features = "\n".join([v
                    for v in extract_input_features(user_query).split("\n")
                    if "NA" not in v and "Not" not in v])
    memory = ConversationBufferMemory(max_token_limit=8000)

        # Initialize GuardRail4LLM with custom harmful keywords and topics
    guardrail = GuardRail4LLM(
        pii_types=["PERSON", "PHONE_NUMBER"],
        harmful_categories=["Toxicity", "Violence", "Hate Speech"],
        custom_harmful_keywords=["explosives", "bomb", "terrorism", "violence"],
        topics = ["Phones", "cell phones", "mobile phones", "smartphones"]
    )

    agent_names = route_agents(user_query).strip()         #LLM capability to route to agent based on user query
    for agent_name in ast.literal_eval(agent_names):
        print("Routing Agents to : " + agent_name)
        if agent_name == "features_tool":

          # Agent 1
          print("Running Feature_agent......")
          cleaned_prompt_features = remove_price_line(prompt_features)
          feature_agent_response = recommendation_agent.run({"input": cleaned_prompt_features})
          filtered_asins = (
              [
                  text.replace("-ASIN : ", "").strip()
                  for text in recommend_similar_features(cleaned_prompt_features).split("\n")
                  if "-ASIN : " in text
                  ]
              )

          # Agent 2
          print("Running Reviews Agent.......")
          for asin in filtered_asins:
            reviews = get_reviews(review_data, asin)[:max_token_limit]
            summarize_reviews_agent.memory = memory
            summary_info = summarize_reviews_agent.run(reviews)
            reviews_summary[asin] = summary_info

            # Agent 3
          print("Running Image Agent.......")
          filtered_mobile_name = ",".join(
              [
                  text.replace("- Mobile Name: ", "").strip()
                  for text in recommend_similar_features(cleaned_prompt_features).split("\n")
                  if "- Mobile Name: " in text
                  ]
              )
          print("filtered_mobile_name : ",filtered_mobile_name)
          generator_tool.func(filtered_mobile_name)

        elif agent_name == "price_tool":

          # Agent 3
          print("Running Price_agent......")
          finetuned_user_query = user_query
          if feature_agent_response:
            filtered_price_prompt = ([text.replace("- ", "").strip() for text in prompt_features.split("\n") if "Price: " in text])
            filtered_asins = ([text.replace("-ASIN : ", "").strip() for text in recommend_similar_features(user_query).split("\n") if "-ASIN : " in text])
            finetuned_user_query = "Suggest products with parent_asin {0} and price {1}".format(filtered_asins, filtered_price_prompt)
            print(finetuned_user_query)

          price_agent_response = price_agent_executor.invoke({"input": finetuned_user_query})['output']

        else:
            return "I'm sorry, I didn't understand your query. Please rephrase and ask again related to cell phones!"

        # LET'S GENERATE SUMMARY
        # Agent4
        summary_agent_response = summary_agent.run(
            f"Feature agent Products: {feature_agent_response}, Price agent Products: {price_agent_response}, Reviews: {reviews_summary}, user_query: {input_user_prompt}"
            " Narrate a user-friendly summary of the recommendations based on context of user_query. "
            " Output should be in dictionary with key {parent_ASIN number}, value : {Product Title, product features, pros, cons and provide recommended reason based on user cotext}. Do not return any extra characters."
        )


    return summary_agent_response

input_user_prompt = "Suggest me mobile 12GB RAM under 350$"

# agent_names = route_agents(user_query).strip()
# print("list of Agents routing" + agent_names)

agents_response_summary = orchestrate_agents(input_user_prompt)
print("final output")
print(agents_response_summary)

Routing Agents to : features_tool
Running Feature_agent......


> Entering new AgentExecutor chain...
I need to find similar products based on the feature of having 12GB of RAM. 
Action: RecommendSimilarFeatures
Action Input: "RAM: 12GB"
Observation: -ASIN : B09TWPRC9Y
- Brand Name: Motorola
- Mobile Name: Motorola Edge 30 Pro
- Processor: 3.0 GHz octa-core processor
- Storage: 256GB internal storage (non-expandable)
- RAM: 12GB
- Camera: Front camera
- Price: $574.5
- Sim Type: Dual-SIM
- Summary of Description: Global version, no warranty, GSM only (not compatible with CDMA carriers), SIM card not included, color: blue
- Battery Life: 4800mAh non-removable battery
- Average Rating: 4.2
- Count of Rating: 77

-ASIN : B08NK5PVB5
- Brand Name: Amazon Renewed
- Mobile Name: OnePlus 8T 5G
- Storage: 256GB
- RAM: 12GB
- Price: $262.0
- Sim Type: Single SIM
- Summary of Description: Professionally inspected and tested pre-owned or refurbished product that works and looks like new. Sold as a

In [91]:
#input_feature_text = "How is weather today?"
input_user_prompt = "Suggest me mobile 12GB RAM under 350$"

agents_response_summary = orchestrate_agents(input_user_prompt)
agents_response_summary

('Off-topic', [])


In [13]:
try:
  agents_response_summary_dict = json.loads(agents_response_summary)
  for asin, product_info in agents_response_summary_dict.items():
    print(f"Product ASIN: {asin}")
    for key, value in product_info.items():
      print(f"  {key}: {value}")
    print("-" * 20)

except json.JSONDecodeError:
  print("Invalid JSON format in agents_response_summary")

Product ASIN: B08683JKTM
  Product Title: Verizon Samsung Galaxy Note10
  product features: {'Storage': '256GB', 'RAM': '12GB LPDDR4X', 'Camera': '16 MP front camera, 10 MP rear camera', 'Price': '$290.0', 'Sim Type': 'Locked to Verizon', 'Battery Life': '4,300 mAh', 'Screen': '6.8" Quad HD Dynamic AMOLED display'}
  pros: ['High-quality display', 'Good camera performance', 'Refurbished product in excellent condition']
  cons: ['Locked to Verizon', 'Limited to 256GB storage']
  recommended reason: This device offers 12GB of RAM and a powerful camera at an affordable price of $290, making it a great choice for users looking for performance and quality under $350.
--------------------
Product ASIN: B09352C4XY
  Product Title: Samsung Galaxy S20 Ultra 5G
  product features: {'Storage': '128GB', 'RAM': '12GB', 'Camera': '108MP rear camera, 40MP front camera', 'Price': '$333.8', 'Sim Type': 'Fully unlocked', 'Battery Life': '5000mAh', 'Screen': '6.9" Dynamic AMOLED 2X, 3200 x 1440'}
  pros:

In [95]:
# prompt: display gradio interface to accept input_user_query and output format as above collected ASIN number as header and dictionary value in square block with black border

# -------------- FINAL GRADIO INTERFACE ---------------

def format_output(user_query):
  prompt_features = "\n".join([v
                    for v in extract_input_features(user_query).split("\n")
                    if "NA" not in v and "Not" not in v])
  agents_response_summary = orchestrate_agents(user_query)

  try:
      agents_response_summary_dict = json.loads(agents_response_summary)
      output_text = ""
      for product_asin, product_info in agents_response_summary_dict.items():
          output_text += f"<div style='border: 2px solid black; padding: 10px; margin-bottom: 10px;'><b>ASIN: {product_asin}</b><br>"
          for key, value in product_info.items():
              output_text += f"{key}: {value}<br>"
          output_text += "</div>"
      folder_path = f"/content/images/"
      image_list = []
      image_names = []
      for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add more image extensions if needed
            image_path = os.path.join(folder_path, filename)
            try:
                image_list.append(image_path)  # Use file path for Gradio to display
                image_names.append(os.path.splitext(filename)[0])  # Exclude file extension
            except Exception as e:
                print(f"Error loading image {filename}: {e}")

      return output_text, image_list, image_names
  except json.JSONDecodeError:
      return "Invalid JSON format in agents_response_summary"



# iface = gr.Interface(
#     fn=format_output,
#     inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
#     outputs=["html",image_list, image_names]
#     title="Product Recommendation",
#     description="Enter a query to get product recommendations.",
# )

iface = gr.Interface(
    fn=format_output,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=[
        gr.HTML(),  # Use gr.HTML for the formatted output
        gr.Gallery(label="Recommended Products"), # Use gr.Gallery for displaying images
        gr.Textbox(label="Image Names") #Use gr.Textbox for Image names
    ],
    title="SPHARK-D BOT Product Recommendation",
    description="Enter a query to get product recommendations.",
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32663eb9b9dcba4e37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
# SENTENCE TRANSFORMER

# ------- Preprocessing Image URLS ---------
def extract_image_data(image_data):
  image_ls = (image_data[1:-1])
  large = ""
  thumbs = ""
  try:
    image_list = ast.literal_eval(image_ls)
    filtered_image_list = [item for item in image_list if item] #remove empty dict
    large = [item.get('large') for item in filtered_image_list if item.get('large')]
    if not large:
      print("empty large")
      thumbs = [item.get('thumb') for item in filtered_image_list if item.get('thumb')]
  except (SyntaxError, ValueError, AttributeError) as e:
    return ""

  return str(large) if large else str(thumbs) #return thumbs if large is empty

def extract_image_urls(image_string):
    try:
        image_urls = ""
        image_list = ast.literal_eval(image_string)
        if isinstance(image_list, list):
            for image_url in image_list:
                return image_url
                # image_urls = image_urls + image_url + ", "
            # return image_urls
        else:
            return ""
    except (SyntaxError, ValueError):
        print("Invalid image string format")
        return ""

# ------- Preprocessing Image URLS ---------


# ---------- TEXT IMAGE EMBEDDINGS ---------------

# Function to download and open an image from a URL
def get_image_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        return image
    except (requests.exceptions.RequestException, IOError):
        return None  # Or handle the error as needed

def generate_image_embeddings(images):
  im_embeddings = []
  for image_url in images:
      image = get_image_from_url(image_url)
      if image:
          # Convert the NumPy array to a PyTorch tensor for model capability
          embeddings = model_transformer.encode(image)
          if isinstance(embeddings, (list, tuple)): # handles multi-image case
              im_embeddings.extend([torch.tensor(emb) for emb in embeddings])
          else:
              im_embeddings.append(torch.tensor(embeddings))
      else:
          print(f"Could not load image from URL: {image_url}")
  return im_embeddings

def truncate_text(text, max_length=74):
    tokens = tokenizer_clip.tokenize(text)
    if len(tokens) > max_length:
        tokens = tokens[:max_length - 1] + [tokenizer_clip.eos_token]  # Add EOS token if truncated
    return tokenizer_clip.convert_tokens_to_string(tokens)

def generate_text_embeddings(feature_data):
  feature_data = feature_data.apply(truncate_text) #VIT limitation
  text_embeddings = model_transformer.encode(feature_data.tolist())
  return text_embeddings

# Embeddings

def transformer_search_feature_text(query,embeddings):
  query_embedding = model_transformer.encode(query)
  results = util.semantic_search(query_embedding, embeddings, top_k=15)
  return results

def retreive_match_images(results, image_data):
  img_list = []
  images_filtered = []
  for result in results:
      try:
        pid=result['corpus_id']
        img_list.append(image_data[pid])
      except:
        img_list.append("")
  print("Image Links retrieved: " + str(img_list))
  for url in img_list:
    if(url):
      response = requests.get(url)
      response.raise_for_status()
      image = Image.open(BytesIO(response.content))
      images_filtered.append(image)
  return images_filtered

def transformer_search_feature_image(query, image_embeddings):
  query_embedding = model_transformer.encode(query)
  results = util.semantic_search(query_embedding, image_embeddings, top_k=15)
  return results

data['feature_extract'] = data['ProductDetails'].astype(str) + ", Price:" + data['price'].astype(str) + ", Average Rating:" + data['average_rating'].astype(str)
data['image_filtered'] = data['images'].apply(extract_image_data)
data['image_urls'] = data['image_filtered'].apply(extract_image_urls)
data = data[data['image_filtered'] != ""]
data = data[data['image_urls'] != ""]
product_data= data.reset_index(drop=True).to_dict(orient='index')
image_data = data["image_urls"]
feature_data = data['feature_extract']

image_embeddings = generate_image_embeddings(image_data)
text_embeddings = generate_text_embeddings(feature_data)

user_query = " 128gb cellphones"
text_results = transformer_search_feature_text(user_query, text_embeddings)
print("Matched Text results: for user query " + user_query + str(text_results))


image_query ="black cellphones"
image_results = transformer_search_feature_image(image_query, image_embeddings)
print("Matched Image results: for user query " + image_query + str(image_results))
image_dt = retreive_match_images(image_results[0], image_data)
[display(image) for image in image_dt]

Output hidden; open in https://colab.research.google.com to view.

In [17]:
# !pip install langchain
# !pip install langchain_community
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import  Document


def create_docs(results):
    docs = []
    for result in results[0]:
        pid = result['corpus_id']
        score = result['score']
        result_string = ''
        result_string += "feature_extract:" + str(product_data[pid]['feature_extract']) +  ';' + \
                          "Parent ASIN:" + str(product_data[pid]['parent_asin']) +  ';' + \
                          "Image URLS:" + str(product_data[pid]['image_urls'])
        doc = Document(page_content=result_string)
        doc.metadata['pid'] = str(pid)
        doc.metadata['score'] = score
        docs.append(doc)
    return docs

def recommend_by_product_features(query):
  results= transformer_search_feature_text(query, text_embeddings)
  docs = create_docs(results)
  response=chain({"input_documents": docs, "question": query},return_only_outputs=True)
  return response['output_text']

def recommend_by_image_color(query):
  results= transformer_search_feature_image(query, image_embeddings)
  docs = create_docs(results)
  response=chain({"input_documents": docs, "question": query},return_only_outputs=True)
  return response['output_text']
# Provide details like parent_asin, RAM, Storage, Camera, Processor, Screen Size, other important information also narrate pros and cons and at the end Recommend why is it good recommendedation given context in your interaction with the customer.

template = """
You are a E-commerce shopping assistant for cellphone products that wants to convert customers based on the information given.

Output should be in dictionary with key parent_asin, value : feature_extract also Narrate with good pros, cons and provide recommended reason based on user cotext. Do not return any extra characters.

Context: {context}

User question:{question}


Your response:"""
prompt = PromptTemplate.from_template(template)

chain = load_qa_chain(llm_gpt3, chain_type="stuff", prompt=prompt)

<ipython-input-17-b331d9bd161a>:53: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm_gpt3, chain_type="stuff", prompt=prompt)


In [18]:
query="Provide good phones in black color with 512GB storage"
print(recommend_by_product_features(query))

<ipython-input-17-b331d9bd161a>:30: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chain({"input_documents": docs, "question": query},return_only_outputs=True)


{
    "Parent ASIN": "B0BY5V8P24",
    "Feature_extract": {
        "Brand Name": "Amazon Renewed",
        "Mobile Name": "Apple iPhone 14 Pro 512GB Deep Purple (Unlocked) (Renewed)",
        "Processor": "A16 Bionic",
        "Storage": "512GB",
        "RAM": "Not specified in the text",
        "Camera": "48MP main camera, cinematic mode in 4K Dolby Vision up to 30 fps",
        "Price": "$1099.66",
        "Sim Type": "Not specified in the text",
        "Summary of Description": "The iPhone 14 Pro is described as powerful, with amazing cameras, a beautiful display, and a dynamic island feature. It is praised for its pocketable form factor, sheer horsepower, and camera capabilities, being considered as close to a perfect phone ever seen.",
        "Battery Life": "Not specified in the text",
        "Average Rating": 4.4,
        "Count of Rating": 127,
        "Screen Details": "6.1-inch Super Retina XDR display featuring always-on promotion, dynamic island feature for interactin

In [28]:
feature_transformer_tool = Tool(
    name = "FeatureTransformerTool",
    func = recommend_by_product_features,
    description = (
        "Recommends products based on features available"
        " Output should be in dictionary with key parent_asin, value : feature_extract also Narrate with good pros, cons and provide recommended reason based on user cotext. Do not return any extra characters."
    )
)

image_transformer_tool = Tool(
    name = "ImageTransformerTool",
    func = recommend_by_image_color,
    description = (
        "Recommend products based on image color. "
        " Output should be in dictionary with key parent_asin, value : feature_extract also Narrate with good pros, cons and provide recommended reason based on user cotext. Do not return any extra characters."
    )
)

recommendation_transformer_agent = initialize_agent(
    tools= [feature_transformer_tool, image_transformer_tool],
    llm= llm_gpt4,
    verbose=True,
    memory=ConversationBufferMemory(),
    handle_parsing_errors=True
)

query="Provide good phones in black color with 256GB storage" #512GB storage in black not found but providing recommendation ab Deep Purple similar
response = recommendation_transformer_agent.run(query)
print(response)



> Entering new AgentExecutor chain...
I need to find smartphones that are available in black color and have 256GB of storage. This will involve searching for features related to smartphones.  
Action: FeatureTransformerTool  
Action Input: {"color": "black", "storage": "256GB"}  
Observation: {
    "parent_asin": "B0BXKWGCSW",
    "feature_extract": {
        "Brand Name": "Samsung",
        "Mobile Name": "Samsung Galaxy S23",
        "Processor": "Fastest mobile processor available",
        "Storage": "256GB",
        "RAM": "Not specified in the provided text",
        "Camera": "50MP high-resolution camera",
        "Price": "$759.99",
        "Sim Type": "Not specified in the provided text",
        "Summary of Description": "The Samsung Galaxy S23 is a powerful smartphone with features like nightography, high camera resolution, fastest mobile processor, long battery life, and adaptive display. It offers convenient collaboration options and quick sharing capabilities.",
       

In [30]:
user_query = "Suggest good phones in black color with 12GB RAM"
recommendation_transformer_agent.run(user_query)
# feature_transformer_tool.run(user_query)



> Entering new AgentExecutor chain...
I need to find phones that match the criteria of having a black color and 12GB of RAM. Since color is a specific feature, I will use the ImageTransformerTool to focus on color recommendations. 
Action: ImageTransformerTool
Action Input: "phones black color 12GB RAM"
Observation: {
    "Parent ASIN": "B0BGG1ZKSP",
    "Feature_extract": {
        "Brand Name": "redmagic",
        "Mobile Name": "redmagic 7s pro smartphone",
        "Processor": "Qualcomm Snapdragon 8gen 1",
        "Storage": "512GB ROM",
        "RAM": "18GB",
        "Camera": {
            "Rear": "64MP + 8MP + 2MP",
            "Front": "16MP"
        },
        "Price": "$849.0",
        "Sim Type": "Dual Nano-SIM",
        "Summary of Description": "Gaming smartphone with 6.8\" FHD+ AMOLED display, 120Hz refresh rate, under-display camera, Qualcomm Snapdragon 8gen 1 processor, 18GB RAM, 512GB ROM, multi-dimensional cooling system, 5G connectivity, 5000mAh battery, 65W fast c

'{\n    "B0BGG1ZKSP": {\n        "Brand Name": "redmagic",\n        "Mobile Name": "redmagic 7s pro smartphone",\n        "Processor": "Qualcomm Snapdragon 8gen 1",\n        "Storage": "512GB ROM",\n        "RAM": "18GB",\n        "Camera": {\n            "Rear": "64MP + 8MP + 2MP",\n            "Front": "16MP"\n        },\n        "Price": "$849.0",\n        "Sim Type": "Dual Nano-SIM",\n        "Summary of Description": "Gaming smartphone with 6.8\\" FHD+ AMOLED display, 120Hz refresh rate, under-display camera, Qualcomm Snapdragon 8gen 1 processor, 18GB RAM, 512GB ROM, multi-dimensional cooling system, 5G connectivity, 5000mAh battery, 65W fast charging, and ultra HD AI quad camera system.",\n        "Battery Life": "5000mAh with 65W fast charging",\n        "Average Rating": 4.2,\n        "Count of Rating": 335,\n        "Screen Details": "6.8 inches FHD+ AMOLED display with 120Hz refresh rate and 2400x1080 pixels resolution."\n    }\n}'

In [96]:
# prompt: gradio interface for accepting user input and generate product recommendation by invoking recommendation_transformer_agent.run(user_query) display in output textbox

import gradio as gr

def format_output(user_query):
    message = gr.update(value="Agents running in progress..")
    try:
      response = recommendation_transformer_agent.run(user_query)
      try:
          output_text = ""
          message = gr.update(value=response)
          return message

      except json.JSONDecodeError:
          message = gr.update(value="Invalid JSON format in agent response")
          return message
    except Exception as e:
        message = gr.update(value=f"An error occurred: {str(e)}")
        return message

# def format_output(user_query):
#     # Assuming recommendation_transformer_agent is defined in the provided code
#     response = recommendation_transformer_agent.run(user_query)
#     return response

iface = gr.Interface(
    fn=format_output,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Textbox(), # Changed output to Textbox for plain text
    title="Product Recommendation by Sentence Transformer",
    description="Enter a query to get product recommendations.",
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d13e81dadd7e964c26.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Response :  200
Output image saved as images/Samsung Galaxy S21 Plus 5G G996U.png.
Generating a new image using DALL-E...


In [ ]:

# ----------------- Additional Testing or already Tested on --------------------------------
# input_product = 'Recommed me Samsung latest mobile phone'
# input_product = "A samsung mobile with 6.5` screen"
# input_product = "samsung galaxy a51"
# input_product = "4GB RAM and 64GB storage smartphone "
# input_product = '6.5" HD display with 90 Hz'
# input_product = 'I want new Iphone with 512GB Storage and screen size more than 6 inch'
# input_product = 'I want new samsung galaxy S22 ultra'
# input_product = 'Recommed me products with 4GB RAM  4000 Battery and Average Rating more than 4'
# input_product = 'Recommed me products with 6GB RAM and 512GB'
# input_product = 'Recommed me products with 8MP Camera and 6GB RAM'
# input_product = 'Recommed me 16GB Memory'
# input_product = 'Best mobile with 6GB Ram'
input_product = "Suggest me mobile with Snapdragon 690"

response = recommendation_agent.run(input_product)
print(response)